**Setting Up DeepSeek-R1 Locally With Ollama**

Ollama simplifies running LLMs locally by handling model downloads, quantization, and execution seamlessly.

Ollama offers a range of DeepSeek R1 models, spanning from 1.5B parameters to the full 671B parameter model. The 671B model is the original DeepSeek-R1, while the smaller models are distilled versions based on Qwen and Llama architectures. If your hardware cannot support the 671B model, you can easily run a smaller version by using the following command and replacing the X below with the parameter size you want (1.5b, 7b, 8b, 14b, 32b, 70b, 671b):

In [ ]:
!sudo apt-get update
!sudo apt-get install -y zstd

In [ ]:
# Install Ollama on Colab VM
!sudo apt-get install -y pciutils
!curl https://ollama.ai/install.sh | sh

In [10]:
# Start the Ollama server
!nohup ollama serve > /content/ollama.log 2>&1 &

In [15]:
# pull the DeepSeek R1 model
!ollama pull deepseek-r1:7b

In [ ]:
!pip install ollama

Once Ollama is installed, use the following script to interact with the model:

In [17]:
import ollama
response = ollama.chat(
    model="deepseek-r1:7b", # Changed model from 32b to 7b for memory compatibility
    messages=[
        {"role": "user", "content": "Explain Newton's second law of motion"},
    ],
)
print(response["message"]["content"])

Newton's second law of motion states that the net force acting on an object is equal to the product of its mass and its acceleration. This relationship can be expressed with the formula:

\[ F = ma \]

Where:
- \( F \) is the net force applied to the object (in Newtons, N),
- \( m \) is the mass of the object (in kilograms, kg),
- \( a \) is the acceleration produced (in meters per second squared, m/s²).

### Key Concepts:
1. **Force**: A push or pull that can change an object's motion by accelerating it.
2. **Mass**: The amount of matter in an object, which determines its inertia—the resistance to changes in motion.
3. **Acceleration**: The rate at which the velocity of an object changes over time.

### Implications:
- For a given force, objects with greater mass experience smaller accelerations. Conversely, for a desired acceleration, more force is needed for heavier objects.
- Net force accounts for all external forces acting on an object; thus, if multiple forces are present, their

The ollama.chat() function takes the model name and a user prompt, processing it as a conversational exchange. The script then extracts and prints the model's response.

In [19]:
response = ollama.chat(
    model="deepseek-r1:7b",
    messages=[
        {
            "role": "user",
            "content": "Tell me a creative story about a talking cat.",
        },
    ],
    options={"temperature": 0.8},  # Set the temperature here
)

print(response["message"]["content"])

Sure! Here's a creative story about a talking cat named Whiskers:

---

Once upon a time in a small, quiet neighborhood on a quaint little island, there lived a mischievous tabby cat named Whiskers who had the most incredible talent. Whiskers could speak in perfect English and could converse with any species he encountered, including people! He was quite popular among the townsfolk because no one else could talk to animals like him.

Whiskers spent his days lounging around the house of a kind old woman named Margaret who lived on the outskirts of town. She adored Whiskers' company and often brought out her collection of toys for him to play with, including a tiny plush rabbit she had rescued from a nearby pound. The rabbit, named Lucky, was a gentle giant among the mice.

But despite his many talents, Whiskers felt neglected by some people in town. He was often mistaken for being a regular feline with no special abilities—but Margaret knew better and made sure to highlight his unique q

**Running a Local Gradio App for RAG With DeepSeek-R1**

Let’s build a simple demo app using Gradio to query and analyze documents with DeepSeek-R1.

**Step 1: Prerequisites**

In [ ]:
!pip install -U langchain chromadb gradio
!pip install -U langchain-community
!pip install --upgrade langchain

In [7]:
import gradio as gr
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
import ollama

LangChain is an open-source framework designed to facilitate the development of applications powered by large language models (LLMs). It offers a suite of tools, components, and interfaces that simplify the construction of LLM-centric applications. With LangChain, it becomes effortless to manage interactions with language models, seamlessly link different components, and incorporate resources such as APIs and databases.

**Step 2: Processing the uploaded PDF**

Once the libraries are imported, we will process the uploaded PDF.

In [2]:
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None

    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, chunk_overlap=100
    )
    chunks = text_splitter.split_documents(data)

    embeddings = OllamaEmbeddings(model="deepseek-r1")
    vectorstore = Chroma.from_documents(
        documents=chunks, embedding=embeddings, persist_directory="./chroma_db"
    )
    retriever = vectorstore.as_retriever()

    return text_splitter, vectorstore, retriever

**Step 3: Combining retrieved document chunks**

Once the embeddings are retrieved, next we need to stitch these together. The combine_docs() function merges multiple retrieved document chunks into a single string.

In [3]:
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

Since retrieval-based models pull relevant excerpts rather than entire documents, this function ensures that the extracted content remains readable and properly formatted before being passed to DeepSeek-R1.

**Step 4: Querying DeepSeek-R1 using Ollama**

Now, our input to the model is ready. Let’s set up DeepSeek R1 using Ollama.




In [4]:
import re

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"

    response = ollama.chat(
        model="deepseek-r1",
        messages=[{"role": "user", "content": formatted_prompt}],
    )

    response_content = response["message"]["content"]

    # Remove content between <think> and </think> tags to remove thinking output
    final_answer = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()

    return final_answer

The ollama_llm() function formats the user’s question and the retrieved document context into a structured prompt. This formatted input is then sent to DeepSeek-R1 via ollama.chat(), which processes the question within the given context and returns a relevant answer. If you need the answer without the model's thinking script, use the strip() function to return the final answer.

**Step 5: The RAG pipeline**

Now we have all the required components, let’s build the RAG pipeline for our demo.




In [5]:
def rag_chain(question, text_splitter, vectorstore, retriever):
    retrieved_docs = retriever.invoke(question)
    formatted_content = combine_docs(retrieved_docs)
    return ollama_llm(question, formatted_content)

The above function first searches the vector store using retriever.invoke(question), returning the most relevant document excerpts. These excerpts are formatted into a structured input using combine_docs function and sent to ollama_llm, ensuring that DeepSeek-R1 generates well-informed answers based on the retrieved content.

**Step 6: Creating the Gradio Interface**

We have our RAG pipeline in place. Now, we can build the Gradio interface locally along with the DeepSeek-R1 model to process PDF input and ask questions related to it.

In [ ]:
!pip install -U ipykernel

In [ ]:
!pip install -U pydantic==2.10.6

In [4]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 48.8 MB/s eta 0:00:00


In [9]:
def ask_question(pdf_bytes, question):
    text_splitter, vectorstore, retriever = process_pdf(pdf_bytes)

    if text_splitter is None:
        return None  # No PDF uploaded

    result = rag_chain(question, text_splitter, vectorstore, retriever)
    return {result}


interface = gr.Interface(
    fn=ask_question,
    inputs=[
        gr.File(label="Upload PDF (optional)"),
        gr.Textbox(label="Ask a question"),
    ],
    outputs="text",
    title="Ask questions about your PDF",
    description="Use DeepSeek-R1 to answer your questions about the uploaded PDF document.",
)

interface.launch(share=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>